# Synthetic Sales Analysis

This notebook processes a synthetic retail dataset and demonstrates a compact, production‑ready workflow. 
It follows a 12‑step scaffold, from loading and profiling to cleaning, feature engineering, aggregation, and serialization.

**Data source**: the synthetic CSV generated earlier in this session, saved as `synthetic_sales.csv`.

## Environment Setup

In [1]:
%pip install --quiet pandas numpy

Note: you may need to restart the kernel to use updated packages.


## 1. Hello, Data!
Load raw CSV, display first 3 rows.

In [2]:
import pandas as pd
import numpy as np
from pathlib import Path

RAW_PATH = Path('data/synthetic_sales.csv')
assert RAW_PATH.exists(), f"File not found: {RAW_PATH}"
df = pd.read_csv(RAW_PATH)
print(df.shape)
df.head(3)

(600, 7)


,date,customer_id,product,price,quantity,coupon_code,shipping_city
0,2024-01-01,CUST8595,Mouse,1426.90,5,SAVE10,Calgary
1,2024-01-02,CUST6983,Tablet,1773.07,3,FREESHIP,Vancouver
2,2024-01-02,CUST3953,Phone,1383.29,2,NaN,Calgary


## 2. Picking the Right Container
Here we are using dictionaries of records, not namedtuples or sets. Dicts are mutable and explicit with keys, which is practical for JSON, schema evolution, and joins. Namedtuples are immutable and verbose to extend. Sets do not preserve field order and are not row‑oriented.

## 3. Implementing Functions and Data structure
Create typed helpers and convert rows into a normalized record dictionary.

In [3]:
from typing import Dict, Any

PRODUCT_TITLE_FIXES = {
    'laptop': 'Laptop', 'phone': 'Phone', 'tablet': 'Tablet',
    'headphones': 'Headphones', 'monitor': 'Monitor', 'keyboard': 'Keyboard', 'mouse': 'Mouse'
}

def normalize_product(name: str) -> str:
    if not isinstance(name, str):
        return 'Unknown'
    key = name.strip().lower()
    return PRODUCT_TITLE_FIXES.get(key, name.strip().title())

def to_record(row: pd.Series) -> Dict[str, Any]:
    return {
        'date': row['date'],
        'customer_id': str(row['customer_id']).strip(),
        'product': normalize_product(row['product']),
        'price': float(row['price']),
        'quantity': int(row['quantity']),
        'coupon_code': str(row['coupon_code']).strip().upper() if pd.notna(row['coupon_code']) else '',
        'shipping_city': str(row['shipping_city']).strip().title()
    }

records = [to_record(r) for _, r in df.iterrows()]
records[:2]

[{'date': '2024-01-01',
  'customer_id': 'CUST8595',
  'product': 'Mouse',
  'price': 1426.9,
  'quantity': 5,
  'coupon_code': 'SAVE10',
  'shipping_city': 'Calgary'},
 {'date': '2024-01-02',
  'customer_id': 'CUST6983',
  'product': 'Tablet',
  'price': 1773.07,
  'quantity': 3,
  'coupon_code': 'FREESHIP',
  'shipping_city': 'Vancouver'}]

## 4. Bulk Loaded
Map DataFrame rows to a list of dictionaries in one call.

In [4]:
records_bulk = df.apply(to_record, axis=1).tolist()
len(records_bulk), type(records_bulk[0])

(600, dict)

## 5. Quick Profiling
Compute min, mean, and max price, and the unique shipping city count.

In [5]:
price_min = df['price'].min()
price_mean = df['price'].mean()
price_max = df['price'].max()
unique_cities = len(set(df['shipping_city']))
print({'price_min': round(price_min,2), 'price_mean': round(price_mean,2), 'price_max': round(price_max,2), 'unique_cities': unique_cities})

{'price_min': np.float64(20.07), 'price_mean': np.float64(1025.93), 'price_max': np.float64(1996.87), 'unique_cities': 8}


## 6. Spotting the Grime
Identify common quality issues in transactional data.

In [7]:
# Dirty data checks
issues = {}

# 1) Missing or blank coupon codes
issues['blank_coupon_codes'] = int((df['coupon_code'].astype(str).str.strip() == '').sum())

# 2) Outlier prices beyond 99th percentile
p99 = df['price'].quantile(0.99)
issues['price_outliers_gt_p99'] = int((df['price'] > p99).sum())

# 3) Non‑conforming product casing
issues['nonstandard_product_casing'] = int((df['product'] != df['product'].str.title()).sum())

# 4) Quantity non‑positive
issues['non_positive_qty'] = int((df['quantity'] <= 0).sum())

# 5) Leading/trailing spaces in customer_id
issues['customer_id_spaces'] = int(df['customer_id'].astype(str).ne(df['customer_id'].astype(str).str.strip()).sum())

issues

{'blank_coupon_codes': 0,
 'price_outliers_gt_p99': 6,
 'nonstandard_product_casing': 0,
 'non_positive_qty': 0,
 'customer_id_spaces': 0}

## 7. Cleaning Rules
Apply fixes inside `clean()`. Show before and after counts.

In [8]:
def clean(raw: pd.DataFrame) -> pd.DataFrame:
    dfc = raw.copy()
    # Parse dates
    dfc['date'] = pd.to_datetime(dfc['date'], errors='coerce')
    # Trim and standardize
    dfc['customer_id'] = dfc['customer_id'].astype(str).str.strip()
    dfc['product'] = dfc['product'].apply(normalize_product)
    dfc['coupon_code'] = dfc['coupon_code'].fillna('').astype(str).str.strip().str.upper()
    dfc['shipping_city'] = dfc['shipping_city'].astype(str).str.strip().str.title()
    # Types and ranges
    dfc['price'] = pd.to_numeric(dfc['price'], errors='coerce').clip(lower=0)
    dfc['quantity'] = pd.to_numeric(dfc['quantity'], errors='coerce').fillna(0).astype(int)
    # Drop impossible rows: null date, non‑positive quantity
    dfc = dfc[dfc['date'].notna()]
    dfc = dfc[dfc['quantity'] > 0]
    return dfc

before = {
    'rows': len(df),
    'null_dates': int(df['date'].isna().sum()),
    'non_positive_qty': int((df['quantity'] <= 0).sum())
}
df_clean = clean(df)
after = {
    'rows': len(df_clean),
    'null_dates': int(df_clean['date'].isna().sum()),
    'non_positive_qty': int((df_clean['quantity'] <= 0).sum())
}
before, after

({'rows': 600, 'null_dates': 0, 'non_positive_qty': 0},
 {'rows': 600, 'null_dates': 0, 'non_positive_qty': 0})

## 8. Transformations
Parse `coupon_code` into a numeric discount.

In [9]:
DISCOUNT_MAP = {
    'SAVE10': 0.10,
    'WELCOME': 0.05,
    'FREESHIP': 0.00  # free shipping assumed, price unchanged
}
df_clean['discount_rate'] = df_clean['coupon_code'].map(DISCOUNT_MAP).fillna(0.0)
df_clean[['coupon_code','discount_rate']].head(10)

,coupon_code,discount_rate
0,SAVE10,0.10
1,FREESHIP,0.00
2,,0.00
3,,0.00
4,,0.00
5,WELCOME,0.05
6,FREESHIP,0.00
7,WELCOME,0.05
8,FREESHIP,0.00
9,,0.00


## 9. Feature Engineering
Add `days_since_purchase` and `net_revenue`.

In [ ]:


# today = pd.Timestamp.utcnow().normalize()
# df_clean['days_since_purchase'] = (today - datetime. df_clean['date']).dt.days
# df_clean['gross_revenue'] = df_clean['price'] * df_clean['quantity']
# df_clean['net_revenue'] = df_clean['gross_revenue'] * (1 - df_clean['discount_rate'])
# df_clean[['date','price','quantity','discount_rate','net_revenue','days_since_purchase']].head(5)

In [11]:
# 1) Ensure 'date' is datetime with UTC timezone
df_clean['date'] = pd.to_datetime(df_clean['date'], utc=True)

# 2) Today at midnight UTC
today = pd.Timestamp.now(tz='UTC').normalize()

# 3) Safe numeric casts
for col in ['price', 'quantity', 'discount_rate']:
    df_clean[col] = pd.to_numeric(df_clean[col], errors='coerce')

# If discount rates come as 10, 15, etc., convert to fractions
pct_mask = df_clean['discount_rate'] > 1
df_clean.loc[pct_mask, 'discount_rate'] = df_clean.loc[pct_mask, 'discount_rate'] / 100.0

# 4) Compute fields
df_clean['days_since_purchase'] = (today - df_clean['date']).dt.days
df_clean['gross_revenue'] = df_clean['price'] * df_clean['quantity']
df_clean['net_revenue'] = df_clean['gross_revenue'] * (1 - df_clean['discount_rate'])

# 5) Preview
df_clean[['date','price','quantity','discount_rate','net_revenue','days_since_purchase']].head(5)


,date,price,quantity,discount_rate,net_revenue,days_since_purchase
0,2024-01-01 00:00:00+00:00,1426.90,5,0.1,6421.05,638
1,2024-01-02 00:00:00+00:00,1773.07,3,0.0,5319.21,637
2,2024-01-02 00:00:00+00:00,1383.29,2,0.0,2766.58,637
3,2024-01-03 00:00:00+00:00,1642.07,2,0.0,3284.14,636
4,2024-01-04 00:00:00+00:00,1733.49,5,0.0,8667.45,635


## 10. Mini‑Aggregation
Revenue per `shipping_city`.

In [12]:
city_rev = df_clean.groupby('shipping_city')['net_revenue'].sum().sort_values(ascending=False)
city_rev_dict = city_rev.round(2).to_dict()
city_rev.head(10), city_rev_dict

(shipping_city
 Winnipeg     270542.9300
 Calgary      257509.4715
 Vancouver    252795.7175
 Ottawa       250271.1010
 Montreal     213205.0335
 Edmonton     194076.3665
 Halifax      181335.2670
 Toronto      178796.7695
 Name: net_revenue, dtype: float64,
 {'Winnipeg': 270542.93,
  'Calgary': 257509.47,
  'Vancouver': 252795.72,
  'Ottawa': 250271.1,
  'Montreal': 213205.03,
  'Edmonton': 194076.37,
  'Halifax': 181335.27,
  'Toronto': 178796.77})

## 11. Serialization Checkpoint
Save cleaned data to JSON.

In [14]:
from pathlib import Path
OUT_JSON = Path('data/cleaned_sales.json')
df_clean.to_json(OUT_JSON, orient='records', lines=False)
str(OUT_JSON)

'data\\cleaned_sales.json'